# 1. Exploração e limpeza das bases de dados 

In [3]:
# importacao das dependencias
import pandas as pd
import numpy as np


## Base de dados "RESULTADOS_MERGED"" 


In [ ]:
# Leitura de duas bases de resultados 
df_resultados = pd.read_csv('../RESULTS_MERGED.csv', low_memory=False)

In [ ]:
# Eliminando colunas desnecessárias para o modelo
df_resultados_limpa = df_resultados.drop(['Unnamed: 0'], axis='columns')
df_resultados_limpa

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA
0,2023-2056234,012001,1,10,NaN,NaN,NaN,2024-05-16 13:41:25
1,2023-2056234,012002,1,10,NaN,NaN,NaN,2024-05-16 14:12:35
2,2023-2056234,012003,1,10,NaN,NaN,NaN,2024-05-16 14:12:27
3,2023-2056234,012004,1,10,NaN,NaN,NaN,2024-05-16 14:11:07
4,2023-2056234,012008,1,10,NaN,NaN,NaN,2024-05-16 14:13:07
...,...,...,...,...,...,...,...,...
11099103,2024-2016184,720100,2,10,Nm,2.0,10.06 ...,2024-03-21 17:48:53
11099104,2024-2016184,720200,2,10,Grad,1.0,21 ...,2024-03-21 17:48:53
11099105,2024-2016184,720200,2,10,Grad,3.0,27.8 ...,2024-03-21 17:48:53
11099106,2024-2016184,720200,2,10,Nm,0.0,10.02 ...,2024-03-21 17:48:53


In [ ]:
# Agrupando por KNR e contando quantos há
df_resultados_limpa.groupby("KNR").count()


,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA
KNR,,,,,,,
2023-2056234,874,874,874,846,846,846,874
2023-5076008,3,3,3,3,3,3,3
2023-5076015,2,2,2,2,2,2,2
2024-0516009,56,56,56,48,56,48,56
2024-0526019,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
2024-2976009,72,72,72,72,72,72,72
2024-2976010,72,72,72,72,72,72,72
2024-2976011,74,74,74,74,74,74,74


In [ ]:
# Transformando a coluna DATA para o tipo datatime 
df_resultados_limpa['DATA'] = pd.to_datetime(df_resultados_limpa['DATA'], errors='coerce')
df_resultados_limpa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32773360 entries, 0 to 11099107
Data columns (total 8 columns):
 #   Column    Dtype         
---  ------    -----         
 0   KNR       object        
 1   NAME      object        
 2   ID        int64         
 3   STATUS    int64         
 4   UNIT      object        
 5   VALUE_ID  float64       
 6   VALUE     object        
 7   DATA      datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 2.2+ GB


In [ ]:
# Convertendo coluna "DATA" para datetime
df_resultados_limpa['DATA'] = pd.to_datetime(df_resultados_limpa['DATA'], errors='coerce')

In [ ]:
# Verificando a tabela de resultados apósa limpeza
df_resultados_limpa

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA
0,2023-2056234,012001,1,10,NaN,NaN,NaN,2024-05-16 13:41:25
1,2023-2056234,012002,1,10,NaN,NaN,NaN,2024-05-16 14:12:35
2,2023-2056234,012003,1,10,NaN,NaN,NaN,2024-05-16 14:12:27
3,2023-2056234,012004,1,10,NaN,NaN,NaN,2024-05-16 14:11:07
4,2023-2056234,012008,1,10,NaN,NaN,NaN,2024-05-16 14:13:07
...,...,...,...,...,...,...,...,...
11099103,2024-2016184,720100,2,10,Nm,2.0,10.06 ...,2024-03-21 17:48:53
11099104,2024-2016184,720200,2,10,Grad,1.0,21 ...,2024-03-21 17:48:53
11099105,2024-2016184,720200,2,10,Grad,3.0,27.8 ...,2024-03-21 17:48:53
11099106,2024-2016184,720200,2,10,Nm,0.0,10.02 ...,2024-03-21 17:48:53


In [ ]:
# Função responsável por transformar os dados 
def aggregate_by_id(df, id_value):
    subset = df[df['ID'] == id_value]
    return subset.groupby('KNR').agg(
        NAME=('NAME', 'count'),  # Conta total de NAME
        SOK=('STATUS', lambda x: (x == 10).sum()),
        SNOK=('STATUS', lambda x: (x == 13).sum()),
        DATA=('DATA', lambda x: (x.max() - x.min()).total_seconds() / (3600*24))
    ).rename(columns={
        'NAME': f'ID{id_value}NAME',
        'SOK': f'ID{id_value}SOK',
        'SNOK': f'ID{id_value}SNOK',
        'DATA': f'ID{id_value}DATA'
    })

# Agregando para cada ID
id1 = aggregate_by_id(df_resultados_limpa, 1)
id2 = aggregate_by_id(df_resultados_limpa, 2)
id718 = aggregate_by_id(df_resultados_limpa, 718)

# Combinando os resultados em um único DataFrame
final_df = id1.join(id2, on='KNR', how='outer').join(id718, on='KNR', how='outer').reset_index()

# Adicionando UNIQUE_ID com valores incrementais

# Reordenando as colunas para o formato desejado
final_df = final_df[['KNR','ID1NAME', 'ID1SOK', 'ID1SNOK', 'ID1DATA', 'ID2NAME', 'ID2SOK', 'ID2SNOK', 'ID2DATA', 'ID718NAME', 'ID718SOK', 'ID718SNOK', 'ID718DATA']]

# Exibindo o resultado
print(final_df)

                KNR  ID1NAME  ID1SOK  ID1SNOK   ID1DATA  ID2NAME  ID2SOK  \
0      2023-2056234     11.0    11.0      0.0  0.036157    776.0   772.0   
1      2023-5076008      NaN     NaN      NaN       NaN      NaN     NaN   
2      2023-5076015      NaN     NaN      NaN       NaN      NaN     NaN   
3      2024-0516009      NaN     NaN      NaN       NaN     16.0    16.0   
4      2024-0526019      NaN     NaN      NaN       NaN      NaN     NaN   
...             ...      ...     ...      ...       ...      ...     ...   
35313  2024-2976009      NaN     NaN      NaN       NaN     72.0    72.0   
35314  2024-2976010      NaN     NaN      NaN       NaN     72.0    72.0   
35315  2024-2976011      NaN     NaN      NaN       NaN     74.0    74.0   
35316  2024-2976012      NaN     NaN      NaN       NaN     82.0    82.0   
35317  2024-2976013      NaN     NaN      NaN       NaN     72.0    72.0   

       ID2SNOK   ID2DATA  ID718NAME  ID718SOK  ID718SNOK  ID718DATA  
0          4.0  4

In [ ]:
final_df

,KNR,ID1NAME,ID1SOK,ID1SNOK,ID1DATA,ID2NAME,ID2SOK,ID2SNOK,ID2DATA,ID718NAME,ID718SOK,ID718SNOK,ID718DATA
0,2023-2056234,11.0,11.0,0.0,0.036157,776.0,772.0,4.0,4.491227,87.0,87.0,0.0,4.422546
1,2023-5076008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2.0,0.037674
2,2023-5076015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,0.018345
3,2024-0516009,NaN,NaN,NaN,NaN,16.0,16.0,0.0,0.019792,40.0,40.0,0.0,3.687118
4,2024-0526019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35313,2024-2976009,NaN,NaN,NaN,NaN,72.0,72.0,0.0,0.045683,NaN,NaN,NaN,NaN
35314,2024-2976010,NaN,NaN,NaN,NaN,72.0,72.0,0.0,0.059190,NaN,NaN,NaN,NaN
35315,2024-2976011,NaN,NaN,NaN,NaN,74.0,74.0,0.0,0.089086,NaN,NaN,NaN,NaN
35316,2024-2976012,NaN,NaN,NaN,NaN,82.0,82.0,0.0,4.293958,NaN,NaN,NaN,NaN


In [ ]:
# Preenchendo os campos que estão em branco após a aplicação da função aggregate_by_id
final_df.fillna(0)

,KNR,ID1NAME,ID1SOK,ID1SNOK,ID1DATA,ID2NAME,ID2SOK,ID2SNOK,ID2DATA,ID718NAME,ID718SOK,ID718SNOK,ID718DATA
0,2023-2056234,11.0,11.0,0.0,0.036157,776.0,772.0,4.0,4.491227,87.0,87.0,0.0,4.422546
1,2023-5076008,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,3.0,1.0,2.0,0.037674
2,2023-5076015,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,2.0,1.0,1.0,0.018345
3,2024-0516009,0.0,0.0,0.0,0.000000,16.0,16.0,0.0,0.019792,40.0,40.0,0.0,3.687118
4,2024-0526019,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35313,2024-2976009,0.0,0.0,0.0,0.000000,72.0,72.0,0.0,0.045683,0.0,0.0,0.0,0.000000
35314,2024-2976010,0.0,0.0,0.0,0.000000,72.0,72.0,0.0,0.059190,0.0,0.0,0.0,0.000000
35315,2024-2976011,0.0,0.0,0.0,0.000000,74.0,74.0,0.0,0.089086,0.0,0.0,0.0,0.000000
35316,2024-2976012,0.0,0.0,0.0,0.000000,82.0,82.0,0.0,4.293958,0.0,0.0,0.0,0.000000


In [ ]:
# Salvando a tabela tratada
final_df.to_csv('tabelas finais/RESULTADOS_FINALMENTE.csv', index=False)